# Figure 1

In [1]:
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from agent.Vampire import SimpleVampire 
from model.VampireModel import VampireModel 

In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be re

In [2]:
MC = 100

# Without reproduction

In [3]:
df_false = pd.DataFrame()
for i in tqdm(range(MC)):
    model = VampireModel(
        n_roots = 10,
        root_size = 15,
        max_iteration = 350,
        hunt_probability = 0.93,
        food_sharing = False,
        reproduction = False,
        vampire_type = SimpleVampire
    )
    model.run_model()
    df_false.insert(loc=i, column=str(i), value=model.datacollector.get_model_vars_dataframe().squeeze())

df_false['mean'] = df_false.mean(axis=1)
df_false['std'] = df_false.std(axis=1)

100%|██████████| 100/100 [00:06<00:00, 16.60it/s]


In [4]:
df_true = pd.DataFrame()
for i in tqdm(range(MC)):
    model = VampireModel(
        n_roots = 10,
        root_size = 15,
        max_iteration = 350, 
        hunt_probability = 0.93,
        food_sharing = True,
        reproduction = False,
        vampire_type = SimpleVampire
    )
    model.run_model()
    df_true.insert(loc=i, column=str(i), value=model.datacollector.get_model_vars_dataframe().squeeze())

df_true['mean'] = df_true.mean(axis=1)
df_true['std'] = df_true.std(axis=1)

100%|██████████| 100/100 [00:12<00:00,  7.77it/s]


# Reproduction

In [5]:
df_false_repro = pd.DataFrame()
for i in tqdm(range(MC)):
    model = VampireModel(
        n_roots = 10,
        root_size = 15,
        max_iteration = 350,
        hunt_probability = 0.93,
        food_sharing = False,
        reproduction = True,
        reproduction_probability = 0.5,
        vampire_type = SimpleVampire
    )
    model.run_model()
    df_false_repro.insert(loc=i, column=str(i), value=model.datacollector.get_model_vars_dataframe().squeeze())

df_false_repro['mean'] = df_false_repro.mean(axis=1)
df_false_repro['std'] = df_false_repro.std(axis=1)

100%|██████████| 100/100 [00:06<00:00, 15.34it/s]


In [6]:
df_true_repro = pd.DataFrame()
for i in tqdm(range(MC)):
    model = VampireModel(
        n_roots = 10,
        root_size = 15,
        max_iteration = 350, 
        hunt_probability = 0.93,
        food_sharing = True,
        reproduction = True,
        reproduction_probability = 0.5,
        vampire_type = SimpleVampire
    )
    model.run_model()
    df_true_repro.insert(loc=i, column=str(i), value=model.datacollector.get_model_vars_dataframe().squeeze())

df_true_repro['mean'] = df_true_repro.mean(axis=1)
df_true_repro['std'] = df_true_repro.std(axis=1)

100%|██████████| 100/100 [00:18<00:00,  5.39it/s]


In [11]:
layout = go.Layout(
    xaxis=dict(
        title='Iteration',
        linecolor="#BCCCDC", 
        showgrid=False  
    ),
    yaxis=dict(
        title='Number of vampires',  
        linecolor="#BCCCDC",  
        showgrid=False,  
    )
)

fig = go.Figure([
    go.Scatter(
        name='With FS and w/o reproduction',
        legendgroup = "Food Sharing",
        x=df_true.index,
        y=df_true['mean'],
        mode='lines',
        line=dict(color='#00FA53', width=4, dash='dot'), 
    ),
    go.Scatter(
        name='With FS and with reproduction',
        legendgroup = "Food Sharing",
        x=df_true_repro.index,
        y=df_true_repro['mean'],
        mode='lines',
        line=dict(color='#00FA53', width=4), 
    ),
    go.Scatter(
        name='Without FS and w/o reproduction',
        legendgroup = "Without Food Sharing",
        x=df_false.index,
        y=df_false['mean'],
        mode='lines',
        line=dict(color='#FA433E', width=4, dash='dot'), 
    ),
    go.Scatter(
        name='Without FS and with reproduction',
        legendgroup = "Without Food Sharing",
        x=df_false_repro.index,
        y=df_false_repro['mean'],
        mode='lines',
        line=dict(color='#FA433E', width=4), 
    )
], layout=layout)

fig.update_layout(
    template="plotly_dark", 
    plot_bgcolor='rgb(0,0,0)', 
    paper_bgcolor='rgb(0,0,0)', 
    font_size=16,
    legend=dict(
        yanchor="bottom",
        y=0.01,
        xanchor="left",
        x=0.01
    )
)
fig.update_yaxes(range=[0, max(
    df_false_repro['mean'].max(), 
    df_true_repro['mean'].max(), 
    df_false['mean'].max(),
    df_true['mean'].max()
)])
fig.to_image(format='png')
fig.write_image('results/food_sharing.png')
# fig.show()
